In [2]:
import numpy as np
import pandas as pd
import itertools as it
import os
import string
import re
import nltk

In [3]:
import gensim
import multiprocessing
import os
# to reload files that are changed automatically
# import utils file - contains some useful functions
import quora_dup_utils as qu
questions_folder_name = "cleaned_data"
google_model_path = '/home/daniel/Desktop/word2vec/word2vec-C-code/data/GoogleNews-vectors-negative300.bin' # path to saved Google word2vec model

# file to store doc2vec hyperparameters and ROC AUC errors
parameters_and_errors_name = "parameters_and_errors.csv"

# number of question pairs to use in training doc2vec
# total number of pairs is currently 404288
num_question_pairs = 404288

D:\Anaconda3_3.5\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [4]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from string import punctuation

data = pd.read_csv('C:/Users/lWX453896/Desktop/python/Quora Question Pairs/train.csv')
data = data.fillna('empty')
data.info()
stop_words = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This']
def text_to_wordlist(text, remove_stop_words=True, stem_words=True):
    # Clean the text, with the option to remove stop_words and to stem words.
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", "", text)
    text = re.sub(r"What's", "", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " America ", text)
    text = re.sub(r" USA ", " America ", text)
    text = re.sub(r" u s ", " America ", text)
    text = re.sub(r" uk ", " England ", text)
    text = re.sub(r" UK ", " England ", text)
    text = re.sub(r"india", "India", text)
    text = re.sub(r"switzerland", "Switzerland", text)
    text = re.sub(r"china", "China", text)
    text = re.sub(r"chinese", "Chinese", text) 
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r"quora", "Quora", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r"KMs", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text) 
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iPhone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"gps", "GPS", text)
    text = re.sub(r"gst", "GST", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"dna", "DNA", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the US", "America", text)
    text = re.sub(r"Astrology", "astrology", text)
    text = re.sub(r"Method", "method", text)
    text = re.sub(r"Find", "find", text) 
    text = re.sub(r"banglore", "Banglore", text)
    text = re.sub(r" J K ", " JK ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

def process_questions(question_list, questions, question_list_name, dataframe):
    '''transform questions and display progress'''
    for question in questions:
        question_list.append(text_to_wordlist(question))
        if len(question_list) % 100000 == 0:
            progress = len(question_list)/len(dataframe) * 100
            print("{} is {}% complete.".format(question_list_name, round(progress, 1)))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
id              404290 non-null int64
qid1            404290 non-null int64
qid2            404290 non-null int64
question1       404290 non-null object
question2       404290 non-null object
is_duplicate    404290 non-null int64
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [5]:
data["question1"] = data.question1.str.lower()
data["question2"] = data.question2.str.lower()




In [6]:
train_question1 = []
process_questions(train_question1, data.question1, 'train_question1', data)


train_question1 is 24.7% complete.
train_question1 is 49.5% complete.
train_question1 is 74.2% complete.
train_question1 is 98.9% complete.


In [7]:

train_question2 = []
process_questions(train_question2, data.question2, 'train_question2', data)

train_question2 is 24.7% complete.
train_question2 is 49.5% complete.
train_question2 is 74.2% complete.
train_question2 is 98.9% complete.


In [8]:
train_question2[:2]

['step by step guid invest in share market',
 'would happen indian govern stole kohinoor koh i noor diamond back']

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
id              404290 non-null int64
qid1            404290 non-null int64
qid2            404290 non-null int64
question1       404290 non-null object
question2       404290 non-null object
is_duplicate    404290 non-null int64
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [46]:
data = pd.read_csv('C:/Users/lWX453896/Desktop/python/Quora Question Pairs/train.csv')

In [47]:
data=   pd.concat([data.id,pd.DataFrame(train_question1),pd.DataFrame(train_question2),data.is_duplicate],axis=1)

In [48]:
data.columns=['id','question1','question2', 'is_duplicate']

In [76]:
list(data.question1.head())

['step by step guid invest in share market in india',
 'stori kohinoor koh i noor diamond',
 'how can i increas speed my internet connect use vpn',
 'whi am i mental veri lone how can i solv it',
 'one dissolv in water quick sugar salt methan carbon di oxid']

In [50]:
cleaned_data = train_question1 +train_question2

In [52]:
cleaned_data[:3]

['step by step guid invest in share market in india',
 'stori kohinoor koh i noor diamond',
 'how can i increas speed my internet connect use vpn']

In [83]:
class LabeledLineSentence(object):
    def __init__(self,question_num, filename):
        self.question_num = question_num
        self.filename = filename 
    def __iter__(self):
        for uid, text_line in enumerate(self.filename):
            yield gensim.models.doc2vec.LabeledSentence(words=text_line.split(), tags=[self.question_num,uid])

# make sure using fast version
assert gensim.models.doc2vec.FAST_VERSION > -1
cores = multiprocessing.cpu_count() # number of cores on computer to use for computations

In [87]:
all_docs = []
sentences = LabeledLineSentence('question1',list(data.question1))
for sentence in sentences:
    all_docs.append(sentence)
sentences = LabeledLineSentence('question2',list(data.question2))
for sentence in sentences:
    all_docs.append(sentence)


In [89]:
len(all_docs)

808580

In [92]:
all_docs[600000:600010]

[LabeledSentence(words=['gastro', 'pub'], tags=['question2', 195710]),
 LabeledSentence(words=['happen', 'in', 'last', 'episod', 'detect', 'conan', 'case', 'close', 'how', 'doe', 'it', 'end'], tags=['question2', 195711]),
 LabeledSentence(words=['it', 'like', 'be', 'summer', 'student', 'at', 'yale', 'how', 'it', 'differ', 'from', 'normal', 'school', 'year'], tags=['question2', 195712]),
 LabeledSentence(words=['how', 'can', 'you', 'improv', 'your', 'communic', 'skill'], tags=['question2', 195713]),
 LabeledSentence(words=['happen', 'when', 'debt', 'ceil', 'higher', 'interest', 'on', 'america', 'debt'], tags=['question2', 195714]),
 LabeledSentence(words=['who', 'invent', 'blow', 'dryer', 'how', 'was', 'it', 'invent'], tags=['question2', 195715]),
 LabeledSentence(words=['how', 'should', 'one', 'chang', 'their', 'diet', 'lose', 'weight'], tags=['question2', 195716]),
 LabeledSentence(words=['sinc', 'airbag', 'add', 'safeti', 'whi', 'don', 't', 'profession', 'race', 'car', 'like', 'in', 

In [95]:
len(list(data.is_duplicate))

404290

In [96]:
# set model parameters parameters

parameters_dict = {

'documents' : all_docs,
'dm' : 0, # use bag-of-words (dbow) model; 1 uses embedding (dmpv) model
'size' : 200, # size of word/doc vectors
'window' : 15, # # max distance between word and neighbor word for word embeddings
'alpha' : .025, # learning rate - use rate in paper
'min_alpha' : 0.0001, # rate from paper
'min_count' : 5, # ignore words with count less than this
'sample' : 1e-5, # how to configure downsampling for high frequency words
'workers' : cores, # number of cores to use
'hs' : 0, # use negative sampling
'negative' : 5, # used in negative sampling
'dbow_words' : 1, # trains word vectors in addition to document vectors in dbow model
'iter' : 3 # recommended number of epochs is ~20 for dbow model on question comparison   

}

In [97]:
# create list of parameters to use in model
from itertools import product
dms = [0]
sizes = [300]
windows = [5]
alphas = [0.025]
min_alphas = [0.0001]
min_counts = [1, 5]
samples = [1e-5]
workers_s = [cores]
hs_s = [0]
negatives = [5]
dbow_words_s = [1]
iters = [150]

# create iterable of all combinations of parameters
params_product = product(dms, sizes, windows, alphas, min_alphas, 
                        min_counts, samples, workers_s, hs_s, negatives, 
                        dbow_words_s, iters)
parameters = [x for x in params_product]

In [ ]:
params_and_errors = []

print "Starting first run of", len(parameters), "runs"
total_time = 0
for run_number, pars in enumerate(parameters): 
    params = {'dm':pars[0], 'size':pars[1], 'window':pars[2], 
              'alpha':pars[3], 'min_alpha':pars[4], 'min_count':pars[5],
              'sample':pars[6], 'workers':pars[7], 'hs':pars[8],
              'negative':pars[9], 'dbow_words':pars[10], 'iter':pars[11]}
    with qu.elapsed_timer() as elapsed:
        model = gensim.models.doc2vec.Doc2Vec(documents=all_docs, **params)
                                                                                                            AUC_value = qu.calculate_AUC(model, doc_names_and_duplicate_class)
        duration = '%.1f' % elapsed()
        # save time to complete computation
        m, s = divmod(float(duration), 60)
        h, m = divmod(m, 60)
        time_string = "%dh %02dm %02ds" % (h, m, s)
        params_and_errors.append((params, AUC_value, time_string))
        total_time += float(duration)
        print 
        print "Completed run number", run_number + 1, "of", len(parameters), "runs total"
        print "AUC score:", round(AUC_value, 4)
        print "Training for this run took", round(float(duration)/60.,1), "minutes"

best_AUC = max([x[1] for x in params_and_errors])
print
print
print "Total training time for all runs:", round(float(total_time)/3600.,2), "hours"
print "Best AUC value:", round(best_AUC, 6)
print "Paramters for best AUC value:", [x[0] for x in params_and_errors if x[1] == best_AUC][0]

# convert params and errors into easy-to-read pandas dataframe
params_df = pd.DataFrame([x[0] for x in params_and_errors])
params_df["AUC"] = pd.Series([x[1] for x in params_and_errors])
params_df["num_doc_pairs"] = pd.Series([len(all_docs) for _ in range(len(params_and_errors))])
params_df["compute_time"] = pd.Series([x[2] for x in params_and_errors])
params_df.sort_values("AUC", ascending=False, inplace=True)

# write parameter values to csv - append if this csv already exists
header=True
if os.path.isfile(parameters_and_errors_name):
    header=False
params_df.to_csv(parameters_and_errors_name, header=header, index=False, mode="a")